In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Ejercicio 2
kartData = pd.DataFrame()

numberOfFiles = 4;
for i in range(numberOfFiles):
    newKartData = pd.read_csv(f'AAKart/ML/Kart{i:02d}.csv')
    
    kartData = pd.concat([kartData, newKartData], axis=0, ignore_index=True)

display(kartData)

,ray1,ray2,ray3,ray4,ray5,kartx,karty,kartz,time,action
0,-1.000000,6.520963,4.611085,6.489805,4.588916,15.98892,0.264488,3.105164,0.100632,ACCELERATE
1,-1.000000,6.520926,4.611081,6.489840,4.588920,15.98892,0.273206,3.105036,0.208055,ACCELERATE
2,-1.000000,6.520916,4.611076,6.489851,4.588925,15.98892,0.275302,3.105015,0.311390,ACCELERATE
3,-1.000000,6.520908,4.611072,6.489858,4.588930,15.98893,0.275676,3.105012,0.401621,ACCELERATE
4,-1.000000,6.520902,4.611067,6.489865,4.588934,15.98893,0.275743,3.105016,0.500187,ACCELERATE
...,...,...,...,...,...,...,...,...,...,...
1416,0.928427,0.835753,1.547613,2.534083,-1.000000,14.98860,0.296702,52.120530,46.001270,ACCELERATE
1417,0.925835,0.842288,1.556550,2.542170,-1.000000,14.44852,0.295907,52.790040,46.100030,ACCELERATE
1418,0.923708,0.840380,1.557029,2.545754,-1.000000,13.86868,0.295823,53.455730,46.200100,ACCELERATE
1419,0.921134,0.836330,1.560062,2.510900,-1.000000,13.24070,0.296903,54.118750,46.302360,ACCELERATE
